In [123]:
import numpy as np
import pandas as pd
import pickle
import datetime
import json
import seaborn as sns
import os

### Variable Information

In [124]:
## Files of interest
##    - HealthKitSamples
##    - HealthKitActivitySummaries (not using for now)
##    - AppleLocationVisits (not using for now)

In [125]:
## Input Directory
directory = "../cumulative_data_811-910"
h_samples_path = "../daily_data/RK.8D1DBFAD.DJW Thesis_20220626-20220627/HealthKitSamples_20220626-20220627.csv"

In [126]:
## Output Directory
out_dir = "../indv_table_exports/"

In [127]:
## Good Subjects
good_subjects = ['01801252-3a7e-4f5f-8b6d-49e8da3902f3',
                 'd26d4b78-7fcf-488e-b687-2d1c93c47b74',
                 '531d7f6d-b880-4a0b-b467-80005a316f1c']

### Participant List

In [128]:
df_participants = pd.read_csv(directory + '/StudyParticipants_20220910.csv')
df_participants["CustomFields"] = df_participants["CustomFields"].apply(json.loads)

participant_list = []
for index, row in df_participants.iterrows():
    if row["CustomFields"]["exp_version"] == "app_pilot_1":
        participant_list.append(row["ParticipantIdentifier"])

### Some handy functions

In [129]:
## Convert time from UTC to ET
def fix_date_to_ET(end_date):
    if pd.to_datetime(end_date, format= '%Y-%m-%d', utc=True).tz_convert('US/Eastern').hour < 5:
        return pd.to_datetime(end_date, format= '%Y-%m-%d', utc=True).tz_convert('US/Eastern').date() - datetime.timedelta(days=1)
    else:
        return pd.to_datetime(end_date, format= '%Y-%m-%d', utc=True).tz_convert('US/Eastern').date()

def fix_columns_by_category(dataframe, categories):
    df = dataframe
    for item in categories:
        if item not in df.columns.to_list():
            df[item] = 'NaN'
    return df

def pivot_df(dataframe, pos: list, col, val):
    df = dataframe
    df = df.pivot_table(index=pos,
                    columns=col, 
                    values=val).reset_index()
    return df

def get_unique_participant_count(dataframe):
    temp = []
    for item in dataframe["ParticipantIdentifier"].to_list():
        if item not in temp:
            temp.append(item)
    return (len(temp))

def get_absent_participants(dataframe, participant_list):
    ## Absent participants
    absent_participants = []

    ## Get participants not found in df_samples
    temp = []
    for item in dataframe["ParticipantIdentifier"].to_list():
        if item not in temp:
            temp.append(item)

    for item in participant_list:
        if item not in temp:
            absent_participants.append(item)
    
    return absent_participants
    
def get_sleep_df(dataframe):
    df_sleep = dataframe
    df_sleep = df_sleep.loc[df_sleep.Type.isin(['SleepAnalysisInterval'])].reset_index(drop=True)

    # Add date column
    df_sleep['SleepDay'] = df_sleep.apply(lambda x: fix_date_to_ET(x.Date), axis=1)
    
    # Calculate and append sleep duration to df
    # Will calculate the duration of each InBed and Asleep Value
    df_sleep['Duration'] = 0
    for i in range(len(df_sleep)):
        m = datetime.datetime.fromisoformat(df_sleep.loc[0, 'Date'])
        n = datetime.datetime.fromisoformat(df_sleep.loc[0, 'StartDate'])
        
        # Convert Time Delta to Hours
        df_sleep.loc[i, 'Duration'] = (m - n).seconds/3600;
    
    # Get sum for each value (InBed, Asleep) for each participant for each SLEEP day
    df_sleep = pd.DataFrame(df_sleep.groupby(['SleepDay','ParticipantIdentifier', 'Value'])['Duration'].sum()).reset_index()
    
    # Make separate columns for InBed and Asleep values
    indices = ['SleepDay', 'ParticipantIdentifier']
    df_sleep = pivot_df(df_sleep, indices, "Value", "Duration")
    
    # Get rid of index name (set to "Value")
    df_sleep.columns.name = None

    # Rename columns for clarity
    df_sleep.rename(columns={'SleepDay': 'StudyDay','Asleep': 'Passive_Asleep','InBed' : 'Passive_InBed'}, inplace=True)
    df_sleep = fix_columns_by_category(df_sleep, ['Passive_Asleep', 'Passive_InBed'])
    
    return df_sleep

def get_heart_rate_df(dataframe, categories):
    df = dataframe
    df_heart = df.loc[df.Type.isin(categories)].reset_index(drop=True)

    # cast Value to numeric
    df_heart.Value = pd.to_numeric(df_heart.Value)

    # Get mean for each type for each participant for each day
    df_heart = pd.DataFrame(df_heart.groupby(['StudyDay','ParticipantIdentifier', 'Type'])['Value'].mean()).reset_index()

    # Make separate columns for type of HeartRate data
    indices = ['StudyDay', 'ParticipantIdentifier']
    df_heart = pivot_df(df_heart, indices, 'Type', 'Value')
    
    # Fix column name
    df_heart = fix_columns_by_category(df_heart, categories)

    # Get rid of index name (set to "Type")
    df_heart.columns.name = None

    # Rename columns
    df_heart.rename(columns={'HeartRateVariability': 'Passive_HeartRate_Variability',
                       'RestingHeartRate' : 'Passive_HeartRate_Resting',
                       'WalkingHeartRateAverage':'Passive_HeartRate_AverageWalking'
                      }, inplace=True)
    return df_heart
    
def get_activity_df(dataframe, mean_categories, sum_categories):
    df = dataframe
    
    # Get activities that need to be summed, and those that need to be averaged in separate DataFrames
    df_mean = df.loc[df.Type.isin(mean_categories)].reset_index(drop=True)
    df_sum = df.loc[df.Type.isin(sum_categories)].reset_index(drop=True)
    
    # cast Value to numeric
    df_mean.Value = pd.to_numeric(df_mean.Value)
    df_sum.Value = pd.to_numeric(df_sum.Value)
    
    # Calculate sums and means
    df_mean = pd.DataFrame(df_mean.groupby(['StudyDay','ParticipantIdentifier', 'Type'])['Value'].mean()).reset_index()
    df_sum = pd.DataFrame(df_sum.groupby(['StudyDay','ParticipantIdentifier', 'Type'])['Value'].sum()).reset_index()
    
    # Pivot take according to activity categories
    # Long to wide
    indices = ['StudyDay', 'ParticipantIdentifier']
    df_mean = pivot_df(df_mean, indices, 'Type', 'Value')
    df_sum = pivot_df(df_sum, indices, 'Type', 'Value')
    
    # Accountfor missing columns
    df_mean = fix_columns_by_category(df_mean, activity_mean_categories)
    df_sum = fix_columns_by_category(df_sum, activity_sum_categories)
    
    # Rename columns
    df_mean.rename(columns={'WalkingSpeed': 'Passive_Activity_AverageWalkingSpeed'
                           }, inplace=True)
    
    df_sum.rename(columns={'ActiveEnergyBurned': 'Passive_Activity_ActiveEnergyBurned',
                        'RestingEnergyBurned' : 'Passive_Activity_RestingEnergyBurned',
                        'DistanceWalkingRunning' :'Passive_Activity_DistanceWalkingRunning',
                        'DistanceCycling' : 'Passive_Activity_DistanceCycling',
                        'AppleStandTime': 'Passive_Activity_AppleStandTime'
                       }, inplace=True)
    
    df_activity = df_sum
    df_activity = df_activity.merge(df_mean, how='left', on=['ParticipantIdentifier', 'StudyDay'])
    return df_activity

def get_other_df(dataframe, categories):
    df = dataframe
    df_other = df.loc[df.Type.isin(categories)].reset_index(drop=True)

    # cast Value to float
    df_other.Value = pd.to_numeric(df_other.Value)

    # Get mean for each type for each participant for each day
    df_other = pd.DataFrame(df_other.groupby(['StudyDay','ParticipantIdentifier', 'Type'])['Value'].mean()).reset_index()

    # convert from long to wide
    df_other = pivot_df(df_other, ['StudyDay', 'ParticipantIdentifier'], 'Type', 'Value')
    
    df_other = fix_columns_by_category(df_other, categories)

    # Get rid of index name (set to "Type")
    df_other.columns.name = None

    # Rename columns
    df_other.rename(columns={'HeadphoneAudioExposure': 'Passive_Audio_HeadphoneExposure',
                       'EnvironmentalAudioExposure' : 'Passive_Audio_EnvironmentalExposure',
                      }, inplace=True)
    return df_other

###  Make HealthKitSamples Dataframe for all results till today

In [131]:
## Make HealthKitSamples Dataframe for all results till today

survey_file_name = ""
path = directory + "/"
for f_name in os.listdir(path):
    if f_name.startswith("HealthKitSamples"):
        survey_file_name = f_name
        break
path = path + '/' + survey_file_name

current_df = pd.read_csv(path) 
current_df["StudyDay"] = current_df.apply(lambda x: fix_date_to_ET(x.Date), axis=1)

In [132]:
## From here we need to get Sleep Data, HeartRate Data, Activity Data, Other Data
df_samples = current_df.loc[current_df["ParticipantIdentifier"].isin(participant_list)]

### Get the absent pilot participants

In [133]:
## Absent participants
absent_participants = []

## Get participants not found in df_samples
temp = []
for item in df_samples["ParticipantIdentifier"].to_list():
    if item not in temp:
        temp.append(item)

for item in participant_list:
    if item not in temp:
        absent_participants.append(item)

### Get Sleep, Heart, Activity, and Other Dataframe

In [134]:
# Get sleep data
df_sleep = get_sleep_df(df_samples)

# Get heart data
heart_categories = ["RestingHeartRate", "WalkingHeartRateAverage", "HeartRateVariability"]
df_heart = get_heart_rate_df(df_samples, heart_categories)

# Get activity data
activity_mean_categories = ['WalkingSpeed']
activity_sum_categories = ['ActiveEnergyBurned', 'RestingEnergyBurned', 'DistanceWalkingRunning',
                  'DistanceCycling', 'AppleStandTime']
df_activity = get_activity_df(df_samples, activity_mean_categories, activity_sum_categories)

# Get other data
other_data_categories = ['HeadphoneAudioExposure', 'EnvironmentalAudioExposure']
df_other = get_other_df(df_samples, other_data_categories)

### Merge the 4 dataframes to get Passive Dataframe

In [135]:
# Merge the 4 Data Frames
## Merge starting with the one with the max participants

sleep = get_unique_participant_count(df_sleep)
heart = get_unique_participant_count(df_heart)
activity = get_unique_participant_count(df_activity)
other = get_unique_participant_count(df_other)

df_passive = df_sleep
if sleep >= heart and sleep >= activity and sleep >= other:
    df_passive = df_sleep
    df_passive = df_passive.merge(df_heart, how='left', on=['ParticipantIdentifier', 'StudyDay'])
    df_passive = df_passive.merge(df_activity, how='left', on=['ParticipantIdentifier', 'StudyDay'])
    df_passive = df_passive.merge(df_other, how='left', on=['ParticipantIdentifier', 'StudyDay'])
elif heart >= sleep and heart >= activity and heart >= other:
    df_passive = df_heart
    df_passive = df_passive.merge(df_sleep, how='left', on=['ParticipantIdentifier', 'StudyDay'])
    df_passive = df_passive.merge(df_activity, how='left', on=['ParticipantIdentifier', 'StudyDay'])
    df_passive = df_passive.merge(df_other, how='left', on=['ParticipantIdentifier', 'StudyDay'])
elif activity >= heart and activity >= sleep and activity >= other:
    df_passive = df_activity
    df_passive = df_passive.merge(df_heart, how='left', on=['ParticipantIdentifier', 'StudyDay'])
    df_passive = df_passive.merge(df_sleep, how='left', on=['ParticipantIdentifier', 'StudyDay'])
    df_passive = df_passive.merge(df_other, how='left', on=['ParticipantIdentifier', 'StudyDay'])
elif other >= heart and other >= activity and other >= sleep:
    df_passive = df_other
    df_passive = df_passive.merge(df_heart, how='left', on=['ParticipantIdentifier', 'StudyDay'])
    df_passive = df_passive.merge(df_activity, how='left', on=['ParticipantIdentifier', 'StudyDay'])
    df_passive = df_passive.merge(df_sleep, how='left', on=['ParticipantIdentifier', 'StudyDay'])

### Displaying absent pilot participant identifiers

In [136]:
sleep_absentees = get_absent_participants(df_sleep, participant_list)
heart_absentees  = get_absent_participants(df_heart, participant_list)
activity_absentees  = get_absent_participants(df_activity, participant_list)
other_absentees  = get_absent_participants(df_other, participant_list)

print("The following pilot participants did not have sleep data")
for item in sleep_absentees:
    print(item)

print("The following pilot participants did not have heart data")
for item in heart_absentees:
    print(item)

print("The following pilot participants did not have activity data")
for item in activity_absentees:
    print(item)

print("The following pilot participants did not have other data")
for item in other_absentees:
    print(item)

The following pilot participants did not have sleep data
48396d3f-058f-4526-b7e7-b260c92774ea
90a04229-610a-4d4f-a5c9-78b388065cfb
d9139666-0f9d-42b3-9ec4-c8d9d0a69aa8
12ab9c18-0f56-401e-8e53-2b9092677fb9
c8892869-a3c8-4dfa-be25-3220d10f48c6
2b10f0e0-ef05-41a6-99cb-a543d0e6cb4e
d39e1b95-ab17-4d80-a25f-10ab76736632
a2a5c70f-4d45-4f4a-bec5-5e2c1776689b
79939a12-9002-468d-a41b-15c3b66d3de5
d7c678bb-10ed-472e-9ef2-dadb65c3ec67
f1ee447d-70b3-4228-8f9b-2f6a8f2391fb
a4e70990-a323-4974-acbc-a90d230b74fa
55305f62-57a2-469e-a2cd-d90f210a6dff
ba7605ab-ed0b-4212-a0cd-bd2e550c6507
64ea1889-52d2-409b-a954-fed089584380
8a5145e5-6977-4beb-b7be-18e4ce6451bb
The following pilot participants did not have heart data
48396d3f-058f-4526-b7e7-b260c92774ea
90a04229-610a-4d4f-a5c9-78b388065cfb
d9139666-0f9d-42b3-9ec4-c8d9d0a69aa8
12ab9c18-0f56-401e-8e53-2b9092677fb9
c8892869-a3c8-4dfa-be25-3220d10f48c6
2b10f0e0-ef05-41a6-99cb-a543d0e6cb4e
d39e1b95-ab17-4d80-a25f-10ab76736632
9346be24-faf8-489d-abe8-4e6be832fe3

### Export passive Dataframe as CSV

In [137]:
## Export CSV File
df_passive.to_csv('passive.csv', index=False)
df_passive

,StudyDay,ParticipantIdentifier,Passive_Activity_ActiveEnergyBurned,Passive_Activity_AppleStandTime,Passive_Activity_DistanceWalkingRunning,Passive_Activity_RestingEnergyBurned,Passive_Activity_DistanceCycling,Passive_Activity_AverageWalkingSpeed,Passive_HeartRate_Variability,Passive_HeartRate_Resting,Passive_HeartRate_AverageWalking,Passive_Asleep,Passive_InBed,Passive_Audio_EnvironmentalExposure,Passive_Audio_HeadphoneExposure
0,2022-08-04,2b10f0e0-ef05-41a6-99cb-a543d0e6cb4e,NaN,NaN,137.910000,NaN,NaN,1.180000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2022-08-04,48396d3f-058f-4526-b7e7-b260c92774ea,NaN,NaN,17.500000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2022-08-04,64ea1889-52d2-409b-a954-fed089584380,NaN,NaN,62.040000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2022-08-04,79939a12-9002-468d-a41b-15c3b66d3de5,NaN,NaN,75.010000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2022-08-04,a4e70990-a323-4974-acbc-a90d230b74fa,NaN,NaN,1697.230000,NaN,NaN,1.318000,NaN,NaN,NaN,NaN,NaN,NaN,90.054043
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
497,2022-09-08,9346be24-faf8-489d-abe8-4e6be832fe3b,NaN,NaN,2067.365538,NaN,NaN,1.071034,NaN,NaN,NaN,NaN,9.196111,NaN,68.237754
498,2022-09-08,ba7605ab-ed0b-4212-a0cd-bd2e550c6507,NaN,NaN,7442.986364,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,60.999503
499,2022-09-08,d7c678bb-10ed-472e-9ef2-dadb65c3ec67,NaN,NaN,474.320000,NaN,NaN,1.120000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
500,2022-09-08,d9d34d8e-046f-475c-baa5-e49552148aff,375.419,131.0,13006.832245,1301.269,NaN,1.366207,36.956080,74.5,117.5,NaN,NaN,NaN,58.461987


In [12]:
####################################Just Testing###########################################
# df_sleep_good
df = df_sleep
df_sleep_good = df[df.ParticipantIdentifier.isin(good_subjects)].reset_index(drop=True)

# df_heart_good
# No data for one of the good participants
df = df_heart
df_heart_good = df[df.ParticipantIdentifier.isin(good_subjects)].reset_index(drop=True)

# df_activity_good
df = df_activity
df_activity_good = df[df.ParticipantIdentifier.isin(good_subjects)].reset_index(drop=True)

# df_other_good
df = df_other
df_other_good = df[df.ParticipantIdentifier.isin(good_subjects)].reset_index(drop=True) 
####################################Just Testing###########################################